In [1]:
import json
import os
import glob
import urllib
import functools
import numpy as np
import pandas as pd

In [2]:
# Downloads contract json files and loads it into DATA

if not os.path.exists("data"):
    os.makedirs("data")
    
for url in (
    "https://techinquiry.org/SiliconValley-Military/subcontracts/microsoft_cloud.json",
    "https://techinquiry.org/SiliconValley-Military/subcontracts/microsoft_noncloud.json",
    "https://techinquiry.org/SiliconValley-Military/subcontracts/amazon.json",
    "https://techinquiry.org/SiliconValley-Military/subcontracts/google.json",
    "https://techinquiry.org/SiliconValley-Military/subcontracts/facebook.json",
    "https://techinquiry.org/SiliconValley-Military/subcontracts/nvidia.json",
    "https://techinquiry.org/SiliconValley-Military/subcontracts/twitter.json",
    "https://techinquiry.org/SiliconValley-Military/subcontracts/palantir.json",
    "https://techinquiry.org/SiliconValley-Military/subcontracts/ida.json",
    "https://techinquiry.org/SiliconValley-Military/subcontracts/mitre.json",
    "https://techinquiry.org/SiliconValley-Military/subcontracts/johns_hopkins.json"
):
    
    path = os.path.join('./data/', url.split('/')[-1])

    if not os.path.isfile(path):
        urllib.request.urlretrieve(url, path)

DATA = {}

for path in glob.glob('./data/*.json'):
    company_name = os.path.splitext(os.path.basename(path))[0]
    
    with open(path, 'r') as f:
        DATA[company_name] = json.load(f)
    


In [20]:
# flatten data in a map of company name to list of contracts
# adds "passthrough_amount" using the "estimated_passthrough_rate"

CONTRACTS = {}

for company in DATA.keys():
    company_contracts = []
    
    for intermediary in DATA[company]:
        for agency in DATA[company][intermediary]:
            for contract in DATA[company][intermediary][agency]:

                # json data uses two different keys -- mistake in original data?
                if "estimated_passthrough_percent" in contract:
                    passthrough_key = "estimated_passthrough_percent"
                else:
                    passthrough_key = "estimated_passthrough_rate"
                                
                company_contracts.append(
                    contract | 
                    { "intermediary": intermediary, "agency":  agency } |
                    { "passthrough_amount": (float(contract[passthrough_key]) / 100) * float(contract["base_and_all_options_value"]) }
                )
    
    CONTRACTS[company] = company_contracts


In [19]:
stats = {}

for company, contracts in CONTRACTS.items():
        stats[company] = sum(map(lambda x: x["passthrough_amount"], contracts))

# sort by amount 
stats = dict(sorted(stats.items(), key=lambda x: x[1], reverse=True))

# format as currency string
{k: '${:0,.2f}'.format(v) for k, v in stats.items()}


{'microsoft_cloud': '$14,906,389,278.88',
 'microsoft_noncloud': '$4,217,538,894.55',
 'amazon': '$163,221,830.57',
 'google': '$32,412,607.80',
 'nvidia': '$7,791,968.60',
 'palantir': '$7,669,369.97',
 'facebook': '$2,895,770.61',
 'twitter': '$805,161.79',
 'mitre': '$741,910.26',
 'ida': '$205,353.67',
 'johns_hopkins': '$27,857.50'}

{'microsoft_cloud': '$19,904,206,668.23',
 'microsoft_noncloud': '$5,647,780,024.40',
 'amazon': '$291,237,334.80',
 'google': '$68,046,317.56',
 'nvidia': '$31,742,561.60',
 'palantir': '$11,798,965.21',
 'facebook': '$6,833,447.03',
 'twitter': '$5,710,057.53',
 'mitre': '$2,967,641.04',
 'ida': '$273,804.89',
 'johns_hopkins': '$55,715.00'}